In [ ]:
from typing import Union, Dict, List, Mapping, Sequence, TypeVar, Generic, Optional, Type, Iterator

TPyJSONBase = Union[str, int, float, bool]

TPyJSON = Union[TPyJSONBase, Mapping[str, 'TPyJSON'], Sequence['TPyJSON']]
from opentelemetry.util import types as ot_types

"""
AttributeValue = Union[
    str,
    bool,
    int,
    float,
    Sequence[str],
    Sequence[bool],
    Sequence[int],
    Sequence[float],
]
"""

from trulens_eval.utils.serial import Lens, GetIndex, GetItem, Step

T = TypeVar("T")
C = TypeVar("C")

ValueOrMappedContainer = Union[T, 'MappedContainer[T]']
ValueOrContainer = Union[T, C]

def Mapped(value: T) -> ValueOrMappedContainer[T]:
    return MappedContainer.make_mapped_container(value=value, lens=Lens(), global_store={}, prefix="trulens_eval@")

class MappedContainer(Generic[T]):
    def __init__(
        self,
        value: Optional[ValueOrContainer[T, C]] = None,
        container_class: Optional[Type[C]] = None,
        global_store: Optional[Dict[str, ot_types.AttributeValue]] = None,
        lens: Optional[Lens] = None,
        prefix: str = "",
    ):
        self.prefix = prefix

        if global_store is None:
            global_store = {}

        self.global_store = global_store

        self.step_store: Dict[Step, ValueOrMappedContainer[T]] = {}

        if lens is None:
            lens = Lens() 
        self.lens = lens

        if value is None:
            if container_class is None:
                raise ValueError("Cannot have a None value and None container class.")
            value = container_class()

        if container_class is None:
            if isinstance(value, TPyJSONBase):
                container_class = None
            else:
                container_class = type(value)

        self.container_class = container_class

        if value is not None:
            self.map(value)

    def _global_key(self, lens: Lens) -> str:
        return self._make_global_key(lens, self.prefix)

    @staticmethod
    def _make_global_key(lens: Lens, prefix: str = "") -> str:
        return prefix + str(lens)

    @staticmethod
    def make_mapped_container(
        value: T,
        global_store: Dict[str, ot_types.AttributeValue],
        lens: Lens, 
        prefix: str = ""
    ) -> ValueOrMappedContainer[T]:
        mtype = MappedContainer.mappedtype_for_value(value)

        if mtype is None:
            global_store[
                MappedContainer._make_global_key(lens, prefix)
            ] = value
            return value
        
        return mtype(
            global_store=global_store,
            value=value,
            lens=lens,
            prefix=prefix
        )

    @staticmethod
    def mappedtype_for_value(value: T) -> Optional[Type['MappedContainer[T]']]:
        if isinstance(value, Mapping):
            return MappedDict
        elif isinstance(value, Sequence):
            return MappedList
        else:
            return None

    def __iter__(self):
        raise TypeError("Cannot iterate over a non-container.")

    def __str__(self):
        return str(self.unmap())

    def __repr__(self):
        return repr(self.unmap())

    def map(self, value: T, lens: Optional[Lens] = None) -> None:
        # print(f"Mapping value under {self.lens}", value, lens)

        if lens is None:
            lens = self.lens

        if isinstance(value, TPyJSONBase):
            self.global_store[self._global_key(lens)] = value

        elif isinstance(value, Sequence):
            for i in range(len(value)):
                step = GetIndex(index=i)
                subcontainer = MappedContainer.make_mapped_container(
                    global_store=self.global_store,
                    lens=self.lens[step],
                    value=value[i],
                    prefix=self.prefix,
                )
                self.step_store[step] = subcontainer

        elif isinstance(value, Mapping):
            for k, v in value.items():
                step = GetItem(item=k)
                subcontainer = MappedContainer.make_mapped_container(
                    global_store=self.global_store,
                    lens=self.lens[step],
                    value=v,
                    prefix=self.prefix
                )
                self.step_store[step] = subcontainer

        else:
            raise TypeError(f"Unexpected type: {type(value)}")

    def __getitem__(self, step: Step) -> Optional[ValueOrMappedContainer[T]]:
        if self.container_class is None:
            raise TypeError("Cannot get item on a non-container.")

        return self.step_store.get(step)

    def __del__(self) -> None:
        # print("Deleting container", self.lens)

        for step, val in list(self.step_store.items()):
            if isinstance(val, MappedContainer):
                val.__del__()
            else:
                lens = self.lens[step]
                del self.global_store[self._global_key(lens)]

            del self.step_store[step]

        if self.container_class is None:
            del self.global_store[self._global_key(self.lens)]

    def __setitem__(self, step: Step, value: T) -> None:
        # print("Setting step", step, value)

        if self.container_class is None:
            raise TypeError("Cannot set item on a non-container.")

        if step in self.step_store:
            del self.step_store[step]

        if isinstance(value, TPyJSONBase):
            mapped_value = value
            self.global_store[self._global_key(self.lens[step])] = value

        elif isinstance(value, (Mapping, Sequence)):
            mapped_value = MappedContainer.make_mapped_container(
                global_store=self.global_store,
                value=value,
                lens=self.lens[step],
                prefix=self.prefix
            )

        else:
            raise TypeError(f"Unexpected type: {type(value)}")

        self.step_store[step] = mapped_value

    def map_value(self, value: T) -> ValueOrMappedContainer[T]:
        return MappedContainer.make_mapped_container(
            global_store=self.global_store, value=value, lens=self.lens, prefix=self.prefix
        )

    def unmap(self) -> ValueOrContainer[T, C]:
        return MappedContainer.unmap_value(self)

    @staticmethod
    def unmap_value(value: ValueOrMappedContainer[T]) -> ValueOrContainer[T, C]:
        if isinstance(value, TPyJSONBase):
            return value

        if isinstance(value, MappedContainer):
            if value.container_class is None:
                return value.global_store[self._global_key(value.lens)]

            container = value.container_class()
            for step, v in value.step_store.items():
                container = step.set(container, MappedContainer.unmap_value(v))
            return container
        else:
            raise TypeError(f"Unexpected type: {type(value)}")

class MappedList(MappedContainer[T], List[T]):
    def __init__(
        self,
        value: Optional[List[T]] = None,
        global_store: Optional[Dict[str, ot_types.AttributeValue]] = None,
        lens: Optional[Lens] = None,
        prefix: str = ""
    ):
        super().__init__(
            value=value,
            global_store=global_store,
            lens=lens,
            container_class=list,
            prefix=prefix
        )
        self.step_store: Dict[GetIndex, ValueOrMappedContainer[T]]

    def __getitem__(self, index: int) -> Optional[ValueOrMappedContainer[T]]:
        if not isinstance(index, int):
            raise TypeError("Expected int index.")
        
        step = GetIndex(index=index)

        return super().__getitem__(step)

    def __setitem__(self, index: int, value: T) -> None:
        # print("List: Setting item", index, value)

        if not isinstance(index, int):
            raise TypeError("Expected integer index.")
        
        step = GetIndex(index=index)

        super().__setitem__(step, value)

    def __iter__(self) -> Iterator[ValueOrMappedContainer[T]]:
        for _, value in self.step_store.items():
            yield value

    def __delitem__(self, index: int) -> None:
        #print("List: Deleting item", index)

        step = GetIndex(index=index)
    
        if not step in self.step_store:
            raise IndexError(f"Index out of range: {index}")

        val = self.step_store[step]
        del val
        del self.step_store[step]

class MappedDict(MappedContainer[T], Dict[str, T]):
    def __init__(
        self,
        value: Optional[Dict[str, T]] = None,
        global_store: Optional[Dict[str, ot_types.AttributeValue]] = None,
        lens: Optional[Lens] = None,
        prefix: str = ""
    ):
        super().__init__(
            value=value,
            global_store=global_store,
            lens=lens,
            container_class=dict,
            prefix=prefix
        )
        self.step_store: Dict[GetItem, ValueOrMappedContainer[T]]

    def __getitem__(self, key: str) -> Optional[ValueOrMappedContainer[T]]:
        if not isinstance(key, str):
            raise TypeError("Expected string key.")
        
        step = GetItem(item=key)

        return super().__getitem__(step)

    def __setitem__(self, key: str, value: T) -> None:
        # print("Dict: Setting item", key, value)
        if not isinstance(key, str):
            raise TypeError("Expected string key.")

        step = GetItem(item=key)

        super().__setitem__(step, value)

    def __iter__(self) -> Iterator[str]:
        for step, value in self.step_store.items():
            yield step.item

    def __delitem__(self, key: str) -> None:
        # print("Dict: Deleting item", key)

        step = GetItem(item=key)

        if step not in self.step_store:
            raise ValueError(f"Key not in mapped dictionary: {step}")

        val = self.step_store[step]
        del val
        del self.step_store[step]


In [ ]:
temp = Mapped({"hello": 1, "true": [1,2,3]})

In [ ]:
temp.global_store

In [ ]:
temp['something'] = {'a': 1, 'b': 2, 'sub': {'c': 3, 'd': 4}}
temp.global_store

In [ ]:
temp

In [ ]:
temp['something']['sub'] = 42
temp

In [ ]:
temp

# Working with Spans

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
repo = Path().cwd().parent.parent.resolve()
sys.path.append(str(repo))

In [ ]:
from pprint import pformat
from pprint import pprint

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
from examples.expositional.end2end_apps.custom_app.custom_retriever import CustomRetriever
import pandas as pd

from trulens_eval import instruments
from trulens_eval.trace.category import Categorizer
from trulens_eval.tru_custom_app import TruCustomApp

In [ ]:
from trulens_eval import Tru
Tru().reset_database()
Tru().start_dashboard(_dev=repo, force=True)

In [ ]:
# Create custom app:
ca = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [ ]:
instruments.Instrument().print_instrumentation()

In [ ]:
ta.print_instrumented()

In [ ]:
with ta as recorder:
    res = ca.respond_to_query(f"hello")

rec = recorder.get()

In [ ]:
rec.calls[0].model_dump()

In [ ]:
spans = Categorizer.spans_of_record(rec)

pd.DataFrame(
    [(
        s.trace_id & 0xff,
        s.name,
        type(s),
        s.span_type,
        s.span_id & 0xff,
        s.parent_span_id & 0xff if s.parent_span_id else 0,
        s.attributes
    ) for s in spans],
    columns=[
        "trace_id",
        "name",
        "type",
        "span_type",
        "span_id",
        "parent_span_id",
        "attributes"
    ],
)


In [ ]:
for span in spans:
    pprint(span)
    pprint(span.model_dump())
    print()